In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 设置请求头部信息，模拟浏览器请求
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
# 设置YachtWorld网站上的搜索条件
search_params = {
    'class': 'sail',
    # 'price': 'USD100000-500000',
    # 'length': '40-50ft'
}
base_url = 'https://www.yachtworld.com'

requests.get(base_url, 
    headers=headers)

<Response [403]>

In [6]:
# 使用 Concurrent.futures 模块中的 ThreadPoolExecutor 类来实现异步请求
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
# pages = 1
pages = 666
req_list = list(range(pages))
# sn = requests.Session()
def transform(i):
    # temp = sn.get(f'https://www.yachtworld.com/boats-for-sale/condition-new/type-sail/?page={i+1}', headers=headers).text
    temp = requests.get(f'https://www.yachtworld.com/boats-for-sale/type-sail/?page={i+1}', headers=headers).text
    # temp = requests.get(f'https://www.yachtworld.com/boats-for-sale/condition-used/type-sail/sort-recommended:desc/?page={i+1}', headers=headers).text
    req_list[i] = BeautifulSoup(temp, 'html.parser')
with ThreadPoolExecutor(max_workers=64) as t:
    tasks = [t.submit(transform, i) for i in range(pages)]
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        pass

100%|██████████| 666/666 [01:37<00:00,  6.82it/s] 


In [7]:
soups = req_list
all(map(lambda x: isinstance(x, BeautifulSoup), soups))

True

In [8]:
from typing import List

def get_urls_from_search_page(soup:List[BeautifulSoup]):
    a = soup.select('#root > div.search > div.flex.flex-row > div.search-right-col > div.pagination-and-results-container.search-display > div.listings-container')
    a = a[0]
    links = [link.get('href') for link  in a.find_all(name='a')]
    return links

links = sum(map(get_urls_from_search_page, soups), [])

IndexError: list index out of range

In [9]:
all(map(lambda x: x.startswith('https://'), links)) # 合法性
len(links) # 数量
# unique的数量
len(set(links))


2188

In [ ]:
with open('old_links.txt', 'w') as f:
    f.write('\n'.join(links))